## Load data and save the Document object

In [1]:
from pathlib import Path
from src.utils import  final_query, fulldocpath, fulllnodespath
import asyncio
import nest_asyncio
nest_asyncio.apply()
# fulldocpath = Path.joinpath(docs_path, "10K_esg")
# fulllnodespath = Path.joinpath(nodes_path)
sq_vec_summary_engine = final_query(fulldocpath, fulllnodespath)

The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs/10K_esg exists. Loading from disk...
The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl exists. Loading from disk...
loading index 10K_esg


In [31]:
# esg_query = "How much was Tesla's Research and Development amount and as a percentage of revenue in 2020?"
# esg_query = "How much did McKinsey invested in strengthening risk management since 2018?"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
# response = esg_query_engine.query(esg_query)
# print(response)


Tesla's Selling, General and Administrative (SG&A) expenses as a percentage of revenue decreased from 8% in 2021 to 5% in 2022.


In [3]:
# sales_query ="What is the total sales in Los Angeles"
# sales_query ="Which Segment sold the most?" 
# response = sales_sql_query_engine.query(sales_query)
# print(response)

In [7]:
# sales_query ="What is the total sales in Los Angeles"
# sales_query ="Which product Segment sold the most using the sales database?"
sales_query ="How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
response = sq_vec_summary_engine.query(sales_query)
print(str(response))

Generated 3 sub questions.
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2022?
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2021?
[Tesla10K_Esg_Report_Query_Engine] Q: What is the difference in Tesla's Selling, General, and Administrative expenses between 2022 and 2021?
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2021 were $4,517 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2022 were $3,946 million.
[Tesla10K_Esg_Report_Query_Engine] A: The difference in Tesla's Selling, General, and Administrative (SG&A) expenses between 2022 and 2021 was a decrease of $571 million, or 13%. Specifically, the SG&A expenses were $3,946 million in 2022, compared to $4,517 million in 2021, representing a decrease of $571 million or 13% year-over-year.
The difference in Tesla's Selli

In [6]:
# esg_query = "How is McKinsey responding to the war in Ukraine in its sustainability effort?"
# esg_query ="How does Deloitte's sustainbaility goals differs from that of McKinsey"
esg_query = "Summarize Deloitte ESg report"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
# esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
response = sq_vec_summary_engine.query(esg_query)
print(response)

Generated 2 sub questions.
[Summary ESG Engine] Q: What are the key highlights from the Deloitte ESG report?
[Tesla10K_Esg_Report_Query_Engine] Q: What are the environmental, social, and governance (ESG) metrics and performance indicators covered in the Deloitte ESG report?
[Summary ESG Engine] A: Based on the information provided in the Deloitte Sustainability Report, here are the key ESG highlights:

Environmental:
- Deloitte reduced its Scope 1 and 2 emissions by 79% in FY2023 compared to the base year.
- Deloitte's fleet transitioned to 39% electric vehicles in FY2023, up from 24% in FY2022.
- Deloitte aims to have 67% of its suppliers set science-based targets by 2025 to reduce emissions in the value chain.

Social: 
- Deloitte launched Deloitte Health Equity Institutes (DHEIs) in Europe, Spanish Latin America, Africa, India, and the US to advance health equity.
- Deloitte signed the Zero Health Gaps Pledge, a global, multi-sector, CEO-level pledge to advance health equity.
- Delo